In [1]:
import os
import obspy
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense
from pathlib import Path
%matplotlib inline
%run helperfunctions.ipynb
os.getcwd()

'/Users/albertkong/grillochallenge'

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
os.chdir('lse_challenge/data')

In [4]:
# Signal List
os.chdir('signal')
signals = []
signal_files = os.listdir()
for sigfile in signal_files[0:300]: #Need to remove index for actual dataset
    tmp_trace = obspy.read(sigfile)
    tmp_trace = tmp_trace.normalize()
    cutlength = np.random.randint(900)
    trace_mat = [tmp_trace[0].data[cutlength:1000+cutlength], tmp_trace[1].data[cutlength:1000+cutlength], tmp_trace[2].data[cutlength:1000+cutlength]]
    mag = np.sqrt(trace_mat[0]**2 + trace_mat[1]**2 + trace_mat[2]**2)
    signals.append([trace_mat[0], trace_mat[1], trace_mat[2], mag, cutlength, 1])
    
    # Negative Transformations
    neg_mat = rot_xyz(trace_mat, 0, -np.pi, 0)
    signals.append([neg_mat[0], neg_mat[1], neg_mat[2], mag, cutlength, 1])
    
    # X, Y, and Z Rotations by Theta
    theta = np.random.uniform()*2*np.pi
    x_mat = rot_x(trace_mat, theta)
    signals.append([x_mat[0], x_mat[1], x_mat[2], mag, cutlength, 1])
    y_mat = rot_y(trace_mat, theta)
    signals.append([y_mat[0], y_mat[1], y_mat[2], mag, cutlength, 1])
    z_mat = rot_z(trace_mat, theta)
    signals.append([z_mat[0], z_mat[1], z_mat[2], mag, cutlength, 1])
    
    # General Rotation by alpha, beta, gamma
    alpha = np.random.uniform()*2*np.pi
    beta = np.random.uniform()*2*np.pi
    gamma = np.random.uniform()*2*np.pi
    rot_mat = rot_xyz(trace_mat, alpha, beta, gamma)
    signals.append([rot_mat[0], rot_mat[1], rot_mat[2], mag, cutlength, 1])
    

# Noise List
os.chdir('../noise')
noise_files = os.listdir()
for noisefile in noise_files[0:300]: #Need to remove index for actual dataset
    tmp_trace = obspy.read(noisefile)
    tmp_trace = tmp_trace.normalize()
    cutlength = np.random.randint(900)
    trace_mat = [tmp_trace[0].data[cutlength:1000+cutlength], tmp_trace[1].data[cutlength:1000+cutlength], tmp_trace[2].data[cutlength:1000+cutlength]]
    mag = np.sqrt(trace_mat[0]**2 + trace_mat[1]**2 + trace_mat[2]**2)
    signals.append([trace_mat[0], trace_mat[1], trace_mat[2], mag, cutlength, 0])
    
    # Negative Transformations
    neg_mat = rot_xyz(trace_mat, 0, -np.pi, 0)
    signals.append([neg_mat[0], neg_mat[1], neg_mat[2], mag, cutlength, 0])
    
    # X, Y, and Z Rotations by Theta
    theta = np.random.uniform()*2*np.pi
    x_mat = rot_x(trace_mat, theta)
    signals.append([x_mat[0], x_mat[1], x_mat[2], mag, cutlength, 0])
    y_mat = rot_y(trace_mat, theta)
    signals.append([y_mat[0], y_mat[1], y_mat[2], mag, cutlength, 0])
    z_mat = rot_z(trace_mat, theta)
    signals.append([z_mat[0], z_mat[1], z_mat[2], mag, cutlength, 0])
    
    # General Rotation by alpha, beta, gamma
    alpha = np.random.uniform()*2*np.pi
    beta = np.random.uniform()*2*np.pi
    gamma = np.random.uniform()*2*np.pi
    rot_mat = rot_xyz(trace_mat, alpha, beta, gamma)
    signals.append([rot_mat[0], rot_mat[1], rot_mat[2], mag, cutlength, 0])
    
os.chdir('../')

sig_df = pd.DataFrame(signals, columns = ["tr_x", "tr_y", "tr_z", "magnitude", "cutlength", "signal"])
sig_df['p_arrival'] = 1000 - sig_df['cutlength']

## Training Neural Network for dectecting whether or not the trace has a signal

In [5]:
# Features and Targets for Identifying the Entire Trace

features = []
targets = []

for index, slice_df in sig_df.iterrows():
    x = slice_df["tr_x"]
    y = slice_df["tr_y"]
    z = slice_df["tr_z"]
    mag = slice_df["magnitude"]
    signal = slice_df["signal"]
    p_arrival = slice_df["p_arrival"]
    d = {"x": x, "y":y, "z":z, "magnitude":mag}
    
    temp_df = pd.DataFrame(data = d)
    temp_df["STA"] = temp_df["magnitude"].rolling(25).mean()
    temp_df["LTA"] = temp_df["magnitude"].rolling(50).mean()
    temp_df["RAV"] = temp_df["STA"]/temp_df["LTA"]
    temp_df["STV"] = temp_df["magnitude"].rolling(25).var()
    temp_df["LTV"] = temp_df["magnitude"].rolling(50).var()
    
    temp_df.dropna(inplace = True)
    
    features.append(temp_df.values)
    targets.append(signal)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size = 0.3)

In [7]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)
feature_shape = X_train[0].shape
feature_shape

(951, 9)

In [8]:
signalmodel = Sequential()
signalmodel.add(LSTM(32, input_shape = feature_shape))
signalmodel.add(Dense(1, activation = 'sigmoid'))
signalmodel.summary()

Metal device set to: Apple M1


2022-03-09 17:53:35.496439: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-09 17:53:35.496652: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32)                5376      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 5,409
Trainable params: 5,409
Non-trainable params: 0
_________________________________________________________________


In [9]:
signalmodel.compile(loss = 'binary_crossentropy', optimizer = "adam", metrics = ['accuracy'])
signalmodel.fit(X_train, y_train, epochs = 40)

Epoch 1/40


2022-03-09 17:53:35.901203: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-09 17:53:36.608118: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-09 17:53:36.751579: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-09 17:53:38.912160: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


79/79 [==============================] - 32s 289ms/step - loss: 0.3518 - accuracy: 0.8496
Epoch 2/40
79/79 [==============================] - 14s 176ms/step - loss: 0.0340 - accuracy: 0.9956
Epoch 3/40
79/79 [==============================] - 14s 176ms/step - loss: 0.0288 - accuracy: 0.9956
Epoch 4/40
79/79 [==============================] - 14s 176ms/step - loss: 0.0356 - accuracy: 0.9940
Epoch 5/40
79/79 [==============================] - 14s 179ms/step - loss: 0.0385 - accuracy: 0.9933
Epoch 6/40
79/79 [==============================] - 14s 177ms/step - loss: 0.9671 - accuracy: 0.5278
Epoch 7/40
79/79 [==============================] - 14s 182ms/step - loss: 0.5926 - accuracy: 0.7774
Epoch 8/40
79/79 [==============================] - 14s 179ms/step - loss: 0.5412 - accuracy: 0.8933
Epoch 9/40
79/79 [==============================] - 14s 177ms/step - loss: 0.4478 - accuracy: 0.8960
Epoch 10/40
79/79 [==============================] - 14s 179ms/step - loss: 0.2660 - accuracy: 0.9381


In [10]:
model_loss, model_accuracy = signalmodel.evaluate(X_test, y_test, verbose = 0)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2022-03-09 18:03:21.319136: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-09 18:03:21.397592: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Loss: 0.018332095816731453, Accuracy: 0.9972221851348877


In [11]:
predictions_test = signalmodel.predict(X_test)
class_test = predictions_test > 0.50
f1_score_test = sklearn.metrics.f1_score(y_test, class_test)
print(f"F1 Score: {f1_score_test}")

2022-03-09 18:03:29.462830: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-09 18:03:29.497152: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


F1 Score: 0.9971777986829727


In [12]:
predictions_train = signalmodel.predict(X_train)
class_train = predictions_train > 0.50
f1_score_train = sklearn.metrics.f1_score(y_train, class_train)
print(f"F1 Score: {f1_score_train}")

F1 Score: 0.9968479117415288


In [13]:
tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_test, class_test).ravel()
(tn, fp, fn, tp)

(547, 2, 1, 530)

## Training Neural Network for dectecting where in the trace the p-wave starts

In [14]:
# Features and Targets for Identifying Windows of Trace
features = []
targets = []
window_step = 40
window_size = 40

for index, slice_df in sig_df.iterrows():
    x = slice_df["tr_x"]
    y = slice_df["tr_y"]
    z = slice_df["tr_z"]
    mag = slice_df["magnitude"]
    signal = slice_df["signal"]
    p_arrival = slice_df["p_arrival"]
    d = {"x": x, "y":y, "z":z, "magnitude":mag}
    
    temp_df = pd.DataFrame(data = d)
    temp_df["STA"] = temp_df["magnitude"].rolling(25).mean()
    temp_df["LTA"] = temp_df["magnitude"].rolling(50).mean()
    temp_df["RAV"] = temp_df["STA"]/temp_df["LTA"]
    temp_df["STV"] = temp_df["magnitude"].rolling(25).var()
    temp_df["LTV"] = temp_df["magnitude"].rolling(50).var()
    
    temp_df.dropna(inplace = True)
    
    start_ind = 0
    end_ind = start_ind + window_size
    
    if signal == 1:

        while end_ind < 950:
            xwindow = temp_df["x"].iloc[start_ind:end_ind]
            ywindow = temp_df["y"].iloc[start_ind:end_ind]
            zwindow = temp_df["z"].iloc[start_ind:end_ind]
            magwindow = temp_df["magnitude"].iloc[start_ind:end_ind]
            #stawindow = temp_df["STA"].iloc[start_ind:end_ind]
            #ltawindow = temp_df["LTA"].iloc[start_ind:end_ind]
            ravwindow = temp_df["RAV"].iloc[start_ind:end_ind]
            stvwindow = temp_df["STV"].iloc[start_ind:end_ind]
            ltvwindow = temp_df["LTV"].iloc[start_ind:end_ind]

            window_data = {"x": xwindow, "y": ywindow, "z": zwindow, "magnitude": magwindow,
                        "RAV": ravwindow, "STV": stvwindow, "LTV": ltvwindow}
            window_df = pd.DataFrame(data = window_data)

            features.append(window_df.values)

            if (p_arrival >= start_ind) and (p_arrival <= end_ind):
                targets.append(1)
            else:
                targets.append(0)

            start_ind += window_step
            end_ind = start_ind + window_size

In [15]:
# For Windows of Trace
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size = 0.3)

In [16]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)
feature_shape = X_train[0].shape
feature_shape

(40, 7)

In [17]:
locmodel = Sequential()
locmodel.add(LSTM(32, input_shape = feature_shape))
locmodel.add(Dense(1, activation = 'sigmoid'))
locmodel.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 32)                5120      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 5,153
Trainable params: 5,153
Non-trainable params: 0
_________________________________________________________________


In [18]:
locmodel.compile(loss = 'binary_crossentropy', optimizer = "adam", metrics = ['accuracy'])
locmodel.fit(X_train, y_train, epochs = 20)

Epoch 1/20


2022-03-09 18:04:03.117446: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-09 18:04:03.239173: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


  5/906 [..............................] - ETA: 12s - loss: 0.5990 - accuracy: 0.9313  

2022-03-09 18:04:03.321070: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


906/906 [==============================] - 13s 14ms/step - loss: 0.1778 - accuracy: 0.9585
Epoch 2/20
906/906 [==============================] - 12s 14ms/step - loss: 0.1506 - accuracy: 0.9604
Epoch 3/20
906/906 [==============================] - 12s 13ms/step - loss: 0.1108 - accuracy: 0.9690
Epoch 4/20
906/906 [==============================] - 12s 14ms/step - loss: 0.0997 - accuracy: 0.9713
Epoch 5/20
906/906 [==============================] - 12s 14ms/step - loss: 0.0940 - accuracy: 0.9729
Epoch 6/20
906/906 [==============================] - 12s 14ms/step - loss: 0.0913 - accuracy: 0.9740
Epoch 7/20
906/906 [==============================] - 12s 13ms/step - loss: 0.0860 - accuracy: 0.9756
Epoch 8/20
906/906 [==============================] - 12s 13ms/step - loss: 0.0880 - accuracy: 0.9744
Epoch 9/20
906/906 [==============================] - 12s 13ms/step - loss: 0.0814 - accuracy: 0.9767
Epoch 10/20
906/906 [==============================] - 13s 14ms/step - loss: 0.0821 - accurac

In [19]:
model_loss, model_accuracy = locmodel.evaluate(X_test, y_test, verbose = 0)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2022-03-09 18:08:08.867636: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-09 18:08:08.920990: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Loss: 0.07124786078929901, Accuracy: 0.9781803488731384


In [20]:
predictions_test = locmodel.predict(X_test)
class_test = predictions_test > 0.25
f1_score_test = sklearn.metrics.f1_score(y_test, class_test)
print(f"F1 Score: {f1_score_test}")

2022-03-09 18:08:11.673357: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-09 18:08:11.707030: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


F1 Score: 0.6425925925925926


In [21]:
predictions_train = locmodel.predict(X_train)
class_train = predictions_train > 0.25
f1_score_train = sklearn.metrics.f1_score(y_train, class_train)
print(f"F1 Score: {f1_score_train}")

F1 Score: 0.6674098848011891


In [35]:
tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_test, class_test).ravel()
(tn, fp, fn, tp)

(11687, 250, 136, 347)

## Exporting Models

In [42]:
os.getcwd()

'/Users/albertkong/grillochallenge/lse_challenge/data'

In [44]:
# Only run if we need to get to root directory
os.chdir('../../')

In [45]:
# Exporting signal model
signalmodel.save('signalmodel')

In [46]:
# Exporting location model
locmodel.save('locmodel')

INFO:tensorflow:Assets written to: locmodel/assets


INFO:tensorflow:Assets written to: locmodel/assets


## Spot checking results

In [50]:
loaded_signalmodel = load_model('signalmodel')

In [51]:
loaded_locmodel = load_model('locmodel')

In [27]:
slice_df = sig_df.iloc[29]

In [28]:
sigfeatures = []
sigtargets = []
locfeatures = []
loctargets = []
window_step = 40
window_size = 40
x = slice_df["tr_x"]
y = slice_df["tr_y"]
z = slice_df["tr_z"]
mag = slice_df["magnitude"]
signal = slice_df["signal"]
p_arrival = slice_df["p_arrival"]
d = {"x": x, "y":y, "z":z, "magnitude":mag}
temp_df = pd.DataFrame(data = d)

In [29]:
temp_df["STA"] = temp_df["magnitude"].rolling(25).mean()
temp_df["LTA"] = temp_df["magnitude"].rolling(50).mean()
temp_df["RAV"] = temp_df["STA"]/temp_df["LTA"]
temp_df["STV"] = temp_df["magnitude"].rolling(25).var()
temp_df["LTV"] = temp_df["magnitude"].rolling(50).var()
    
temp_df.dropna(inplace = True)
sigfeatures.append(temp_df.values)
sigtargets.append(signal)

In [52]:
loaded_signalmodel.predict(np.array(sigfeatures))

2022-03-09 18:34:11.938515: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-09 18:34:11.992965: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


array([[ 0.9943389]], dtype=float32)

In [31]:
sigtargets

[1]

In [32]:
start_ind = 0
end_ind = start_ind + window_size
    

while end_ind < 950:
    xwindow = temp_df["x"].iloc[start_ind:end_ind]
    ywindow = temp_df["y"].iloc[start_ind:end_ind]
    zwindow = temp_df["z"].iloc[start_ind:end_ind]
    magwindow = temp_df["magnitude"].iloc[start_ind:end_ind]
    ravwindow = temp_df["RAV"].iloc[start_ind:end_ind]
    stvwindow = temp_df["STV"].iloc[start_ind:end_ind]
    ltvwindow = temp_df["LTV"].iloc[start_ind:end_ind]

    window_data = {"x": xwindow, "y": ywindow, "z": zwindow, "magnitude": magwindow,
                    "RAV": ravwindow, "STV": stvwindow, "LTV": ltvwindow}
        
    window_df = pd.DataFrame(data = window_data)

    locfeatures.append(window_df.values)

    if (p_arrival >= start_ind) and (p_arrival <= end_ind):
        loctargets.append(1)
    else:
        loctargets.append(0)

    start_ind += window_step
    end_ind = start_ind + window_size

In [53]:
loaded_locmodel.predict(np.array(locfeatures))

2022-03-09 18:34:16.942859: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-09 18:34:16.977427: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


array([[ 0.00166442],
       [ 0.00244572],
       [ 0.00734585],
       [ 0.02321231],
       [ 0.9868595 ],
       [ 0.02351769],
       [ 0.06364629],
       [ 0.05422267],
       [ 0.13535184],
       [ 0.02546414],
       [ 0.00627017],
       [ 0.00219755],
       [ 0.00206448],
       [ 0.00352113],
       [ 0.00327192],
       [ 0.00239588],
       [ 0.0017788 ],
       [ 0.00211299],
       [ 0.00183   ],
       [ 0.00209795],
       [ 0.0019537 ],
       [ 0.00187928],
       [ 0.0014608 ]], dtype=float32)

In [34]:
loctargets

[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
## Pseudo Coding Test Script
"""
1) Take in trace file
2) Normalize data
3) Calculate Fields for signalmodel
4) Predict whether or not the trace contains a signal

If YES:
5) Calculate Fields for locmodel
6) Predict where the pwave is
7) Take the max probability
8) Convert to a UTC timestamp
9) Return UTC timestamp

If NO:
5) Return None
"""